<a href="https://www.inove.com.ar"><img src="https://raw.githubusercontent.com/InoveAlumnos/dataset_analytics_python/master/images/PA%20Banner.png" width="1000" align="center"></a>


# BinaryEncoding

Explicación de la nota mencionada en el proyecto de partidos de futbol<br>
v1.1

In [1]:
import numpy as np
import pandas as pd

In [4]:
# Dataframe de ejemplo
df = pd.DataFrame({
      "local": ["Argentina", "Holanda", "Estados Unidos",
                      "Argentina", "Estados Unidos",
                      "Inglaterra", "Estados Unidos"],
      "visitante": ["Holanda", "Estados Unidos", "Holanda",
                      "Inglaterra", "Estados Unidos",
                      "Inglaterra", "Estados Unidos"]
        }
      )
df

,local,visitante
0,Argentina,Holanda
1,Holanda,Estados Unidos
2,Estados Unidos,Holanda
3,Argentina,Inglaterra
4,Estados Unidos,Estados Unidos
5,Inglaterra,Inglaterra
6,Estados Unidos,Estados Unidos


In [27]:
# Lo primero que se puede notar es que las columnas no tienen exactamente los
# mismos paises, en la coulmna de visitante falta por ejemplo Argentina
# El encoder se debe de crear con todas las distintas posibilidades

# 1) Crear un vector contenga todos los elementos 
vector_total = pd.concat([df['local'], df['visitante']])

# 2) De ese vector con todos los posibles elementos, extraer los unicos
vector_unicos = vector_total.unique()
vector_unicos

array(['Argentina', 'Holanda', 'Estados Unidos', 'Inglaterra'],
      dtype=object)

In [50]:
class BinaryEncoder():
    def __init__(self):
        self.n_bits = 0
    
    def fit(self, data):
        if data.__class__.__name__ == list.__name__:
          self.n_bits = int(np.ceil(np.log2(max(data)+1))) + 1
        elif type(data).__module__ == pd.core.series.__name__:          
          self.n_bits = int(np.ceil(np.log2(data.max()+1))) + 1
        elif type(data).__module__ == np.__name__:
          self.n_bits = int(np.ceil(np.log2(data.max()+1))) + 1
        else:
          raise ValueError("Allowed list, numpy or pandas serie data")

    def transform(self, data):
        binary_encoding = [format(int(x+1), '0'+str(self.n_bits)+'b') for x in data]
        binary_encoding_split = [list(x) for x in binary_encoding]
        binary_encoding_header = ['b'+str(x) for x in reversed(range(self.n_bits))]
        return pd.DataFrame(binary_encoding_split, columns=binary_encoding_header, dtype=int)

    def fit_transform(self, data):
        self.fit(data)
        return self.transform(data)

In [66]:
from sklearn.preprocessing import LabelEncoder
# Antes de poder correr el BinaryEncoder debemos transformar
# las variables categoricas a numeros que las representen (nº de clases)

# Entrenar al label encoder:
le = LabelEncoder()
label_encoding = le.fit_transform(vector_unicos.reshape(-1, 1))

# Entrenar el BinaryEncoder
binary_encoder = BinaryEncoder()
binary_encoder.fit(label_encoding)

Traduccion del label encoding ['Argentina' 'Estados Unidos' 'Holanda' 'Inglaterra']


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [73]:
# Ahora que tenemos los dos encoders entrenados podemos transformar
# cada columna del dataframe por separado
# Local
local_label_encoding = le.transform(df['local'])
local_b = binary_encoder.transform(local_label_encoding)
local_b = local_b.add_prefix('local_')
local_b

,local_b2,local_b1,local_b0
0,0,0,1
1,0,1,1
2,0,1,0
3,0,0,1
4,0,1,0
5,1,0,0
6,0,1,0


In [72]:
# Visitante
local_label_encoding = le.transform(df['visitante'])
visitante_b = binary_encoder.transform(local_label_encoding)
visitante_b = visitante_b.add_prefix('visitante_')
visitante_b

,visitante_b2,visitante_b1,visitante_b0
0,0,1,1
1,0,1,0
2,0,1,1
3,1,0,0
4,0,1,0
5,1,0,0
6,0,1,0


In [77]:
# Ahora resta construir un dataframe utilizando las 
# columnas transformadas en vez de las originales
df2 = df.copy()
# Agregar las nuevas columnas
df2 = df2.join(local_b)
df2 = df2.join(visitante_b)
# Del nuevo dataframe eliminamos las columnas viejas
df2 = df2.drop('visitante', axis=1)
df2 = df2.drop('local', axis=1)
df2

,local_b2,local_b1,local_b0,visitante_b2,visitante_b1,visitante_b0
0,0,0,1,0,1,1
1,0,1,1,0,1,0
2,0,1,0,0,1,1
3,0,0,1,1,0,0
4,0,1,0,0,1,0
5,1,0,0,1,0,0
6,0,1,0,0,1,0


In [78]:
# Este dataframe lo podemos guardar en una base de datos o exportar como CSV
df2.to_csv('dataset_encoder.csv')